In [1]:
# Importando bibliotecas

import pandas as pd
import re
import numpy as np
import inflection
import streamlit as st
from PIL import Image
import folium
from streamlit_folium import folium_static

# configurando para mostrar todas as colunas
pd.set_option('display.max_columns', None)

# dando nome ao DataFrame

df = pd.read_csv('zomato.csv')

# Criação de Funções


In [2]:
# Renomear as colunas do DataFrame
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df
# Colocando a função para rodar 
df1 = rename_columns(df)

In [3]:
# Preenchimento do nome dos países
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

# Colocando a função para rodar 
# Criando uma nova coluna chamada country e colocando ela no lugar da country_code
df1.insert(loc=df1.columns.get_loc("country_code"), column="country", value=df1["country_code"].apply(country_name))
# Excluindo a coluna country_code
df1.drop("country_code", axis=1, inplace=True)

In [4]:
# Criação do Tipo de Categoria de Comida
def create_price_type(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

# Colocando a função para rodar  

df1['price_type'] = df1['price_range'].apply(create_price_type)
# removendo a coluna price_range
df1.drop("price_range", axis=1, inplace=True)

# removendo a coluna price_type
price_type = df1.pop("price_type")

# inserindo a coluna price_type na nova posição
df1.insert(10, "price_type", price_type)

In [5]:
# Criação do nome das Cores
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

def color_name(color_code):
    return COLORS[color_code]

# Colocando a função para rodar 
df1.insert(loc=df1.columns.get_loc("rating_color"), column="color_name", value=df1["rating_color"].apply(color_name))


# Limpeza dos dados 

In [6]:
# categorizarodos os restaurantes somente por um tipo de culinária
df1["cuisines"] = df1.loc[:, "cuisines"].apply(lambda x: x.split(",")[0] if isinstance(x, str) else x)

In [7]:
# excluir coluna de Menu 
df1.drop("switch_to_order_menu", axis=1, inplace=True)

In [8]:
# limpando dados duplicados

df1.drop_duplicates(subset="restaurant_id", keep="first", inplace=True)
# Excluido o index 
df1.reset_index(drop=True, inplace=True)

In [9]:
# Removendo Linhas que tem valores NaN 
df1.dropna(inplace=True)

# seleciona linhas que contém as strings 'Drinks Only' ou 'Mineira' na coluna 'cuisines'
linhas_remover = df1[df1['cuisines'].str.contains('Drinks Only|Mineira')]

# remove as linhas selecionadas
df1.drop(linhas_remover.index, inplace=True)

# Convertendo as moedas para dolar e criando uma coluna nova 

In [10]:
# Convertendo as moedas para dolar 
conversion_rates = {
    'Botswana Pula(P)': 13.61,
    'Brazilian Real(R$)': 4.97,
    'Emirati Diram(AED)': 3.67,
    'Indian Rupees(Rs.)': 82.81,
    'Indonesian Rupiah(IDR)': 14894.60,
    'NewZealand($)': 1.62,
    'Pounds(£)': 0.80,
    'Qatari Rial(QR)': 3.64,
    'Rand(R)': 19.20,
    'Sri Lankan Rupee(LKR)': 305.01,
    'Turkish Lira(TL)': 19.87
}

def dollar(df):
    df1['price_in_dollar'] = df1[['currency', 'average_cost_for_two']].apply(lambda x: 
        x['average_cost_for_two'] if x['currency'] == 'Dollar($)' else x['average_cost_for_two'] / conversion_rates.get(x['currency'], 1), axis=1).round(2)
    return df1

#  Aplicando a Função 
df1 = dollar(df)

# Verificando se tem algum valor fora do normal 

In [11]:
df1.loc[df1['country'] == 'Australia', 'average_cost_for_two'].unique()


array([      70,       45,      160,       35,       40,      110,
             85,      100,       50,       96,       65,       80,
             90, 25000017,       75,       60,       95,       30,
             55,      130,      150,      140,      120,      250,
            200,       20,      165,      170,        0,      105,
            139,       44,      115], dtype=int64)

In [12]:
# Procurando a linha que estar com o valor muito suspeito no data frame 
df1.loc[(df1['country'] == 'Australia') & (df1['average_cost_for_two'] == 25000017), :]


,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,price_type,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,color_name,rating_color,rating_text,votes,price_in_dollar
356,16608070,d'Arry's Verandah Restaurant,Australia,Adelaide,"Osborn Rd, McLaren Vale",McLaren Vale,McLaren Vale,138.545242,-35.198372,Modern Australian,cheap,25000017,Dollar($),1,0,0,4.7,darkgreen,3F7E00,Excellent,203,25000017.0


In [13]:
# substituindo o valor pelo valor "Medio" dos pratos do restaurante 
df1.loc[(df1['country'] == 'Australia') & (df1['average_cost_for_two'] == 25000017), 'average_cost_for_two'] = 185

In [14]:
#verificando se ocorreu a correção do valor 
df1.loc[(df1['country'] == 'Australia') & (df1['average_cost_for_two'] == 185), :]


,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,price_type,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,color_name,rating_color,rating_text,votes,price_in_dollar
356,16608070,d'Arry's Verandah Restaurant,Australia,Adelaide,"Osborn Rd, McLaren Vale",McLaren Vale,McLaren Vale,138.545242,-35.198372,Modern Australian,cheap,185,Dollar($),1,0,0,4.7,darkgreen,3F7E00,Excellent,203,25000017.0


In [18]:
# Mudando o valor do Price_in_dollar
df1.loc[(df1['country'] == 'Australia') & (df1['price_in_dollar'] == 25000017.0), 'price_in_dollar'] = 185

In [19]:
df1.loc[(df1['country'] == 'Australia') & (df1['average_cost_for_two'] == 185), :]

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,price_type,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,color_name,rating_color,rating_text,votes,price_in_dollar
356,16608070,d'Arry's Verandah Restaurant,Australia,Adelaide,"Osborn Rd, McLaren Vale",McLaren Vale,McLaren Vale,138.545242,-35.198372,Modern Australian,cheap,185,Dollar($),1,0,0,4.7,darkgreen,3F7E00,Excellent,203,185.0


# Exportanto o arquivo tratado

In [20]:
df1.to_csv('zomato_tratado.csv', sep=',', index=False)